# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42) # 훈련 세트 중 20%를 다시 떼어 검증 세트로 만듦

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


## 교차 검증

In [7]:
from sklearn.model_selection import cross_validate # 교차 검증

scores = cross_validate(dt, train_input, train_target) # 기본값으로 5-폴드 교차 검증을 함
print(scores)

{'fit_time': array([0.01302743, 0.01005721, 0.01093102, 0.01039124, 0.00997639]), 'score_time': array([0.00123739, 0.00105524, 0.00107336, 0.00102425, 0.00110364]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score'])) # 교차 검증의 평균

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold # 분류 모델에서 사용하는 분할기

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) 
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # n_splits : 몇 폴드 교차 검증을 할지, shuffle : 교차 검증 시 훈련 세트를 섞음
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [11]:
from sklearn.model_selection import GridSearchCV # 그리드 서치

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # n_jobs : 사용할 CPU 코어 수, -1일 경우 모든 코어 사용

In [13]:
gs.fit(train_input, train_target) # 교차 검증을 통해 하이퍼파라미터를 찾은 뒤 전체 훈련 세트로 모델 다시 만듦

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_ # 앞서 찾은 최적의 하이퍼파라미터를 가진 모델이 저장되어 있음
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
print(gs.best_params_) # 최적의 하이퍼파라미터가 저장되어 있음

{'min_impurity_decrease': 0.0001}


In [16]:
print(gs.cv_results_['mean_test_score']) # 각 교차검증으로 얻은 점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index]) # 그 중 가장 큰 값의 하이퍼파라미터 = best_params

{'min_impurity_decrease': 0.0001}


In [18]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), # 노드를 분할하기 위한 불순도 감소 최소량
          'max_depth': range(5, 20, 1), # 트리의 깊이 제한
          'min_samples_split': range(2, 100, 10) # 노드를 나누기 위한 최소 샘플 수
          }

In [19]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [20]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [21]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [22]:
from scipy.stats import uniform, randint # 주어진 범위에서 고르게 값을 뽑음

In [23]:
rgen = randint(0, 10)
rgen.rvs(10)

array([7, 3, 5, 2, 0, 6, 5, 9, 2, 8])

In [24]:
np.unique(rgen.rvs(1000), return_counts=True) # 샘플 수가 늘어날수록 고르게 분포되고 있음을 확인

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([101, 116, 105, 112,  86, 102,  79,  90, 116,  93]))

In [25]:
ugen = uniform(0, 1) # 0과 1 사이의 실수
ugen.rvs(10)

array([0.42564184, 0.61017822, 0.37935544, 0.27328306, 0.55379526,
       0.73718345, 0.80802164, 0.18756866, 0.27525532, 0.14591785])

In [26]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [27]:
from sklearn.model_selection import RandomizedSearchCV # 연속적인 실수 값에서 최적의 값을 찾고자 할때 유용

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42) # n_iter : n번 반복
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d52693d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d5269610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d49b4dd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d49b4390>},
                   random_state=42)

In [28]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [29]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [30]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [31]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d52693d0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d5269610>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d49b4dd0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fe9d49b4390>},
                   random_state=42)

In [32]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


splitter가 best(각 노드에서 최선의 분할)에서 random(무작위로 분할한 후 가장 좋은 것을 고름)으로 변화하자 성능이 내려간 것을 확인